In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
pip install peft

In [ ]:
pip install trl

In [4]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
# from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = '42dot/42dot_LLM-SFT-1.3B'

# model.bin 파일의 경로
new_model = "/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/practice/Xwena"

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model)


model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using device: cpu


In [10]:
import torch

# 모델을 CPU로 이동
model = model.to('cpu')

# 모델의 파라미터를 FP32로 변경
model = model.float()

# 텍스트 입력
text = "유성페인트의 환경 오염에 대한 예시를 알려주세요."

# 토큰화
tokens = tokenizer(text, return_tensors="pt")

# 텍스트 생성 설정
max_length = 400  # 생성할 텍스트의 최대 길이
num_return_sequences = 1  # 생성할 텍스트 시퀀스의 수

# 텍스트 생성
with torch.no_grad():
    generated_ids = model.generate(
        tokens["input_ids"],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(f"Generated text: {generated_text}")

Generated text: 유성페인트의 환경 오염에 대한 예시를 알려주세요.
유성페인트는 환경 오염을 일으킬 수 있는 몇 가지 예시가 있습니다.
1. 유성페인트의 분해로 인해 대기 중의 유해한 물질이 생성될 수 있습니다. 예를 들어, 유성페인트의 분해로 인해 포름알데히드와 같은 유해 물질이 생성될 수 있습니다.
2. 유성페인트는 자연 환경에 해를 끼칠 수 있습니다. 예를 들어, 유성페인트의 분해로 인해 토양과 물이 오염될 수 있습니다. 또한, 유성페인트의 분해로 인해 나무와 같은 식물에도 손상을 줄 수 있습니다.
3. 유성페인트는 수질 오염을 일으킬 수 있습니다. 유성페인트의 분해로 인해 물 속에 유해한 물질이 생성될 수 있으며, 이는 수생 생물에게 해를 끼칠 수 있습니다.
4. 유성페인트는 건축물의 내구성을 약화시킬 수 있습니다. 유성페인트의 분해로 인해 건축물의 표면에 균열이 생길 수 있으며, 이는 건물의 안전성을 위협할 수 있습니다.
5. 유성페인트는 실내 공기 오염을 일으킬 수 있습니다. 유성페인트의 분해로 인해 실내 공기에 유해한 물질이 생성될 수 있으며, 이는 호흡기 질환과 같은 건강 문제를 유발할 수 있습니다.
6. 유성페인트는 재활용이 어렵습니다. 유성페인트는 분해되지 않고 그대로 폐기물로 처리되기 때문에 재활용이 어렵습니다.
7. 유성페인트는 자연 환경에 대한 부정적인 영향을 최소화하기 위해 친환경적인 대안이 필요합니다. 예를 들어, 유성페인트 대신 친환경적인 페인트를 사용하는 것이 환경 오염을 줄일 수 있습니다.
